##1. Introduction/Business Problem
This project deals with discussing the neighborhoods of   Mumbai, India. This project is aimed to help Business people looking forward to start new Cafe,Restaurants, Hotels, etc. in Mumbai, India by analysing the neighbourbood for existing businesses and choosing the most suitable location. 

The Foursquare API is used to access the venues in the neighborhoods. Then they are clustered based on their venues using Data Science Techniques. Here the k-means clustering algorithm is used to achieve the task. The optimal number of clusters can be obtained using sum of squared distance. Folium visualization library can be used to visualize the clusters superimposed on the map of Mumbai. These clusters can be analyzed to help small scale business owners select a suitable location for their need.

In this I have taken Coffee Shop / Cafe as an example.

##2. Data Sections


1. Importing Libraries

In [3]:
!pip install geocoder
!pip install folium

import pandas as pd
import requests
import numpy as np
import geocoder
import folium
import requests 
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
import xml
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
from bs4 import BeautifulSoup
from geopy.extra.rate_limiter import RateLimiter

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

     |████████████████████████████████| 102kB 4.0MB/s 


###2.   What data is used?
The data is obtained from wikipedia: Neighbourhood of Mumbai<br>
Web-scrapped using soup. 

In [4]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Mumbai").text
soup = BeautifulSoup(data, 'lxml')
textList = []
neighborhoodList = []

# append the data into the list
neighborhoodList.clear()

for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
    
df = pd.DataFrame({"Neighborhood": neighborhoodList})
df1 = df.iloc[1:]
mumbai_df = df1.reset_index(drop=True)
mumbai_df.head()

,Neighborhood
0,Aarey Forest
1,Agripada
2,Altamount Road
3,"Amboli, Mumbai"
4,Amrut Nagar


###3. Geo-coordinates

In [5]:
# Geographical coordinates of neighborhoods

# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

coords = [ get_latlng(neighborhood) for neighborhood in mumbai_df["Neighborhood"].tolist() ]

df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

# merge the coordinates into the original dataframe
mumbai_df['Latitude'] = df_coords['Latitude']
mumbai_df['Longitude'] = df_coords['Longitude']


# check the neighborhoods and the coordinates
print(mumbai_df.shape)
mumbai_df.head()

(134, 3)


,Neighborhood,Latitude,Longitude
0,Aarey Forest,19.138660,72.884250
1,Agripada,18.976280,72.826150
2,Altamount Road,18.964334,72.807842
3,"Amboli, Mumbai",18.940170,72.834830
4,Amrut Nagar,19.145160,72.846740


###4. Data visualisation using folium

In [6]:
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
# print('The geograpical coordinate of Mumbai is {}, {}.'.format(latitude, longitude))

# create map of Berlin using latitude and longitude values
map_mumbai = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(mumbai_df['Latitude'], mumbai_df['Longitude'], mumbai_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_mumbai)  
    
map_mumbai